In [15]:
import csv
import re
import statistics
import random

In [2]:
# task-2 is a subset of task-1
with open('data/task-1/train.csv') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

In [5]:
def get_vocabulary(rows):
    vocab = set()
    for row in rows:
        sent = row['original'].replace('<', '').replace('/>', '')
        words = sent.split()
        vocab.update(words)
        vocab.add(row['edit'])
    return vocab

In [28]:
def get_edited_text(row):
    orig = row['original']
    edited = re.sub(r"<.+/>", row['edit'], orig)
    return edited

In [35]:
def get_std(row):
    grades = [int(x) for x in row['grades']]
    return statistics.pstdev(grades)

In [43]:
sorted_rows = sorted(rows, key=lambda x: (x['meanGrade'], -get_std(x)), reverse=True)

In [16]:
def get_subset_glove(rows):
    from tqdm import tqdm
    vocab = get_vocabulary(rows)
    with open('data/vocab.txt', 'w') as f:
        for word in vocab:
            f.write(word)
            f.write('\n')
    with open('glove.840B.300d.txt') as f, open('glove.trimmed.300d.txt', 'w') as of:
        for line in tqdm(f):
            word = line.split(' ', 1)[0]
            if word in vocab:
                of.write(line)

In [28]:
random.shuffle(rows)
n = len(rows)
# train-test split
training_set = rows[:int(n*0.8)]
test_set = rows[int(n*0.8):]

# train-dev split
dev_set = training_set[int(len(training_set)*0.8):]
train_set = training_set[:int(len(training_set)*0.8)]

for fn, set_rows in zip(['data/train.csv', 'data/test.csv', 'data/dev.csv'], [train_set, test_set, dev_set]):
    with open(fn, 'w') as f:
        writer = csv.DictWriter(f, fieldnames=set_rows[0].keys())
        writer.writeheader()
        writer.writerows(set_rows)
